## import model

In [1]:
import matplotlib
# Force matplotlib to not use any Xwindows backend.
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.models import Sequential, model_from_json, load_model
from keras.layers.core import Dense, Dropout, Flatten, Activation, SpatialDropout2D, Reshape, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU, PReLU, LeakyReLU
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Convolution1D
from keras import layers
from keras.layers import LSTM ,Embedding ,SimpleRNN,CuDNNLSTM
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from scipy.io import wavfile
import pdb
import scipy.io
# import librosa
import os
from os.path import join as ojoin
# os.environ["CUDA_VISIBLE_DEVICES"]="1"
import time  
import numpy as np
import numpy.matlib
import argparse
import random
# import theano
# import theano.tensor as T
import tensorflow as tf
from keras.callbacks import TensorBoard
import keras.backend.tensorflow_backend as KTF
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [2]:
#設定使用GPU 檢查是否有抓到
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.99 #使用45%記憶體
set_session(tf.Session(config=config))

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


## 定義讀檔路徑function

In [3]:
def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.

    for root, directories, files in os.walk(directory):

        for filename in files:
            if filename.endswith('.wav'):
            # Join the two strings in order to form the full filepath.
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)  # Add it to the list.
                # pdb.set_trace()
    file_paths.sort()
    return file_paths 

In [4]:
mixed_file=get_filepaths('mixed_all_snr/')
cleaned_file=get_filepaths('clean')

## 切割data and test 

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mixed_file, cleaned_file, test_size=0.33, random_state=42)    

In [7]:
Train_Noisy_lists=X_train
Train_Clean_paths= y_train

Test_Noisy_lists  = X_test
Test_Clean_paths = y_test
          
Num_testdata=len(Test_Noisy_lists)   
Num_traindata=len(Train_Noisy_lists)

## np.shape(noisy)[0] <88800

In [8]:
a=np.array([1,2,3])
b=np.array([0,0,0])

x=np.hstack((a,b))
x,x.shape

(array([1, 2, 3, 0, 0, 0]), (6,))

In [9]:
rate, noisy = wavfile.read(X_train[0])

In [10]:
np.shape(noisy)[0]

48449

In [11]:
noisy=np.reshape(noisy,(1,np.shape(noisy)[0],1))

In [12]:
noisy.shape

(1, 48449, 1)

In [13]:
def train_data_generator(noisy_list, clean_path):
    index=0
    while True:
        #noisy, rate  = librosa.load(noisy_list[index],sr=16000) 
        # print(noisy_list[index],index)
        # pdb.set_trace()
        rate, noisy = wavfile.read(noisy_list[index])
        
        # while noisy.shape[0]/16000.>7: # Audio length <7s or OOM 
        #     index += 1
        #     rate, noisy = wavfile.read(noisy_list[index])

        noisy=noisy.astype('float32')         
        if len(noisy.shape)==2:
            noisy=(noisy[:,0]+noisy[:,1])/2  
        noisy=np.reshape(noisy,(1,np.shape(noisy)[0],1))

        #clean, rate  =librosa.load(clean_list[clean_wav_list.index(noisy_wav_list[index])],sr=16000)         
        rate, clean = wavfile.read(clean_path[index])
        clean=clean.astype('float32')  
        clean=clean/2**15
            
        clean=np.reshape(clean,(1,np.shape(clean)[0],1))
        
        index += 1
        if index == len(noisy_list):
            index = 0

#             permute = list(range(len(noisy_list)))
#             random.shuffle(permute)
#             noisy_list=shuffle_list(noisy_list,permute)

        yield noisy, clean

def val_data_generator(noisy_list, clean_path):
    index=0
    while True:
         #noisy, rate  = librosa.load(noisy_list[index],sr=16000)       
        rate, noisy = wavfile.read(noisy_list[index])
        noisy=noisy.astype('float32')         
        if len(noisy.shape)==2:
            noisy=(noisy[:,0]+noisy[:,1])/2       
#         noisy=noisy/np.max(abs(noisy))
#         if np.shape(noisy)[0] <88800:
#             add_zeros = np.zeros(88800-np.shape(noisy)[0])
#             noisy=np.hstack((noisy,add_zeros))
#         noisy=np.reshape(noisy,(1,np.shape(noisy)[0],1))

        noisy=np.reshape(noisy,(1,np.shape(noisy)[0],1))
          
        rate, clean = wavfile.read(clean_path[index])
        clean=clean.astype('float32')  
        if len(clean.shape)==2:
            clean=(clean[:,0]+clean[:,1])/2
        clean=clean/2**15
#         if np.shape(clean)[0] <88800:
#             add_zeros = np.zeros(88800-np.shape(noisy)[0])
#             clean=np.hstack((clean,add_zeros))

        clean=np.reshape(clean,(1,np.shape(clean)[0],1))
        

        index += 1
        if index == len(noisy_list):
            index = 0
          
        yield noisy, clean 

## 建立模型

In [14]:
start_time = time.time()

model = Sequential()
# model.add(LSTM(32,return_sequences=True,input_shape=(None,1)))
# model.add(LSTM(32,return_sequences=True)) # 返回维度为 32 的向量序列
# model.add(Dense(1,activation='tanh'))

model.add(CuDNNLSTM(32,return_sequences=True,input_shape=(None,1)))
model.add(CuDNNLSTM(32,return_sequences=True)) # 返回维度为 32 的向量序列
model.add(Dense(1,activation='tanh'))
model.summary()

W0709 09:05:57.425796 140509956875776 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/Keras-2.2.4-py3.7.egg/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, None, 32)          4480      
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, None, 32)          8448      
_________________________________________________________________
dense_1 (Dense)              (None, None, 1)           33        
Total params: 12,961
Trainable params: 12,961
Non-trainable params: 0
_________________________________________________________________


## 訓練開始

In [15]:
epoch=2
batch_size=1
model.compile(loss='mse', optimizer='adam')
    
with open('{}.json'.format('firsttry'),'w') as f:    # save the model
    f.write(model.to_json()) 
checkpointer = ModelCheckpoint(filepath='{}.hdf5'.format('firsttry'), verbose=1, save_best_only=True, mode='min')  

print ('training...')

g1 = train_data_generator(Train_Noisy_lists, Train_Clean_paths)
print()
g2 = val_data_generator(Test_Noisy_lists, Test_Clean_paths)

tbCallBack = TensorBoard(log_dir='./logs',  # log 目录
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
#                  batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=True, # 是否可视化梯度直方图
                 write_images=True,# 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)   
# hist=model.fit_generator(g1,steps_per_epoch=10000, epochs=epoch, verbose=1,
#                             validation_data=g2,
#                             validation_steps=g2,
#                             max_queue_size=1, 
#                             workers=1,
#                             )
hist=model.fit_generator(g1,
                         samples_per_epoch=Num_traindata,
#                         steps_per_epoch=10000, 
                        epochs=epoch, 
                        verbose=1,
                        validation_data=g2,
                        nb_val_samples=Num_testdata,
                        max_q_size=1, 
                        nb_worker=16,
                        use_multiprocessing=True
#                          pickle_safe=False,
                         )   

W0709 09:05:59.311595 140509956875776 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/Keras-2.2.4-py3.7.egg/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=5, verbose=1, validation_data=<generator..., use_multiprocessing=True, steps_per_epoch=3859, validation_step

training...

Epoch 1/5


/usr/local/lib/python3.7/site-packages/Keras-2.2.4-py3.7.egg/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


3859/3859 [==============================] - 10888s 3s/step - loss: 2.2973e-04 - val_loss: 2.3801e-04
Epoch 2/5
3859/3859 [==============================] - 10530s 3s/step - loss: 1.9741e-04 - val_loss: 2.0186e-04
Epoch 3/5
1633/3859 [===========>..................] - ETA: 1:22:39 - loss: 1.6042e-04

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3859/3859 [==============================] - 10476s 3s/step - loss: 2.2300e-04 - val_loss: 3.6632e-04
Epoch 5/5
1114/3859 [=======>......................] - ETA: 1:40:06 - loss: 2.5968e-04

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 畫Loss圖

In [20]:
# # plotting the learning curve
TrainERR=hist.history['loss']
ValidERR=hist.history['val_loss']
print ('@%f, Minimun error:%f, at iteration: %i' % (hist.history['val_loss'][epoch-1], np.min(np.asarray(ValidERR)),np.argmin(np.asarray(ValidERR))+1))
# print 'drawing the training process...'
plt.figure(4)
plt.plot(range(1,epoch+1),TrainERR,'b',label='TrainERR')
plt.plot(range(1,epoch+1),ValidERR,'r',label='ValidERR')
plt.xlim([1,epoch])
plt.legend()
plt.xlabel('epoch')
plt.ylabel('error')
plt.grid(True)
plt.show()
plt.savefig('Learning_curve_{}.png'.format('FCN_firsttry'), dpi=150)


end_time = time.time()
print ('The code for this file ran for %.2fm' % ((end_time - start_time) / 60.))


@0.000214, Minimun error:0.000202, at iteration: 2
The code for this file ran for 982.94m


## 開始predict囉 並轉成音檔

In [17]:
maxv = np.iinfo(np.int16).max 
for path in Test_Noisy_lists[:10]: # Ex: /mnt/hd-02/avse/testing/noisy/engine/1dB/1.wav
    S=path.split('/') 
    # noise=S[-3]
    # dB=S[-2]
    wave_name=S[1]
    
    rate, noisy = wavfile.read(path)
    noisy=noisy.astype('float32')
    if len(noisy.shape)==2:
        noisy=(noisy[:,0]+noisy[:,1])/2             
#     noisy=noisy/np.max(abs(noisy))
    noisy=np.reshape(noisy,(1,np.shape(noisy)[0],1))
    enhanced=np.squeeze(model.predict(noisy, verbose=0, batch_size=batch_size))
#     enhanced=enhanced/np.max(abs(enhanced))
    enhanced=enhanced* maxv
    enhanced_2=enhanced.astype('int16')
#     enhanced=enhanced/2**15
    wavfile.write(os.path.join("c", wave_name),16000,(enhanced).astype(np.int16))
#     librosa.output.write_wav(os.path.join("/FCN", wave_name), (enhanced* maxv).astype(np.int16), 16000)

#     librosa.output.write_wav(os.path.join("FCN_enhanced_MSE",noise, dB, wave_name), (enhanced* maxv).astype(np.int16), 16000)
